In [3]:
Alpha_Name = "Original_Input_Moving_10_Shift_5"
start_date = "2020-06-01"
end_date = "2021-01-01"
sequence = 5
LR = 0.001
epoch_num = 30
feature_num = 108

In [ ]:
import time
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import DataLoader
import AlphaNet 
from AlphaNet.Models import AlphaNet_LSTM_V1
model_path = "/home/wuwenjun/Alpha_Factor/" + Alpha_Name + "/" + "%s_%s" %(start_date,end_date) + "/"
if os.path.exists(model_path) == False:
    os.makedirs(model_path)
    print(model_path)
data_path = "/home/ShareFolder/feature_platform/ti0/wuwenjun/#Data_lib/"



# Train Loader

trainloader = DataLoader()
trainloader.load_data_from_file(alpha_name = Alpha_Name,end_date = start_date,data_path=data_path)
train_loader = trainloader.to_torch_DataLoader(sequence = sequence,shuffle=True)

# Model Loader
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss_function = nn.MSELoss()
model = AlphaNet_LSTM_V1(feature_num, 64)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
model_loader = AlphaNet.Model_Loader(model = model,device=device)
print(model_loader.model)

# Training
model = model_loader.fit_transform(train_loader,optimizer,loss_function,epoch_num,save_path = model_path)

optimizer = optim.Adam(model.parameters(), lr=LR/10)
model = model_loader.transform(optimizer = optimizer, epoch_num = epoch_num, method="best")

optimizer = optim.Adam(model.parameters(), lr=LR/100)
model = model_loader.transform(optimizer = optimizer, epoch_num = epoch_num, method="best")

# Test
testloader = DataLoader()
testloader.load_data_from_file(alpha_name = Alpha_Name,start_date = start_date,end_date = end_date)
test_loader = testloader.to_torch_DataLoader(sequence = 3,shuffle=False)
pred = model_loader.pred(test_loader)

# convert to standard daily and back test
final = pd.concat([testloader.target.reset_index(),pred],axis=1)
final.rename(columns={0:Alpha_Name,'ticker': 'symbol'},inplace=True)
final.to_parquet(model_path + "result.par")
convert_to_standard_daily_feature_csv(Alpha_name, final, output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun')

# back test

pd.set_option('expand_frame_repr', False)
configs = namespace.load_namespace(r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template')
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")


FT.load_feature_from_file(alpha_list, "2020-01-01", end_date, universe='Investable',timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()
FT.get_intersection_ic(feature_name=Alpha_Name, truncate_fold=None, method='spearman',period=('1d', '3d', '5d'))
ic_flag, trading_direction = FT.test_ic(Alpha_Name, verbose=False)
df, df_all = FT.get_ic_summary_by_month(num=6)

if trading_direction == -1:
    negative = True
else:
    negative = False


# save txt
f = open(model_path + 'back_test.txt','a')
print("*"*100,end="\n"*3,file=f)
print(Alpha_Name,end = "\n",file=f)
print(start_date,end = "\n",file=f)
print(end_date,end = "\n",file=f)
print(sequence,end = "\n",file=f)
print(LR,end = "\n",file=f)
print(epoch_num,end = "\n",file=f)
print(feature_num,end = "\n"*3,file=f)
print(df,end="\n"*3,file = f)
a = FT.get_top_return(Alpha_Name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                        tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
print(a[1],end = "\n"*3+"*"*100 ,file = f)
f.close()

  2%|▏         | 31/1317 [00:00<00:06, 198.66it/s]

/home/wuwenjun/Alpha_Factor/Original_Input_Moving_10_Shift_5/2020-06-01_2021-01-01/


  0%|          | 0/30 [00:00<?, ?it/s]

AlphaNet_LSTM_V1(
  (batch): BatchNorm1d(108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(108, 64, num_layers=5, batch_first=True, bidirectional=True)
  (batch2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Learning Rate is : 0.001



0it [00:00, ?it/s]
1it [00:00,  3.11it/s]
7it [00:00, 19.95it/s]
15it [00:00, 36.89it/s]
25it [00:00, 53.55it/s]
33it [00:00, 61.11it/s]
42it [00:00, 68.62it/s]
50it [00:00, 71.51it/s]
58it [00:01, 72.99it/s]
66it [00:01, 75.00it/s]
76it [00:01, 81.44it/s]
85it [00:01, 75.74it/s]
93it [00:01, 68.71it/s]
101it [00:01, 70.03it/s]
109it [00:01, 70.17it/s]
117it [00:01, 66.74it/s]
126it [00:02, 71.83it/s]
135it [00:02, 75.24it/s]
143it [00:02, 75.23it/s]
153it [00:02, 79.95it/s]
162it [00:02, 78.22it/s]
172it [00:02, 82.16it/s]
181it [00:02, 84.21it/s]
191it [00:02, 88.02it/s]
200it [00:02, 86.72it/s]
209it [00:02, 84.42it/s]
218it [00:03, 83.65it/s]
227it [00:03, 76.99it/s]
235it [00:03, 76.42it/s]
243it [00:03, 77.22it/s]
251it [00:03, 77.95it/s]
261it [00:03, 83.37it/s]
271it [00:03, 85.73it/s]
281it [00:03, 87.84it/s]
292it [00:03, 91.62it/s]
302it [00:04, 89.32it/s]
311it [00:04, 84.54it/s]
320it [00:04, 84.91it/s]
329it [00:04, 82.00it/s]
338it [00:04, 79.27it/s]
346it [00:04, 77.85

...

Epoch:  26  loss:  0.5558974097735362



0it [00:00, ?it/s]
1it [00:02,  2.05s/it]
9it [00:02,  5.63it/s]
15it [00:02, 10.19it/s]
20it [00:03,  6.97it/s]
27it [00:03, 11.24it/s]
33it [00:04,  8.52it/s]
40it [00:04, 12.48it/s]
49it [00:05, 12.59it/s]
57it [00:05, 17.54it/s]
65it [00:06, 15.56it/s]
72it [00:06, 19.95it/s]
81it [00:06, 24.17it/s]
88it [00:06, 26.54it/s]
93it [00:06, 22.58it/s]
97it [00:07, 23.91it/s]
107it [00:07, 34.71it/s]
113it [00:07, 19.64it/s]
120it [00:07, 23.50it/s]
125it [00:08, 15.39it/s]
128it [00:08, 16.12it/s]
135it [00:08, 22.12it/s]
141it [00:09, 14.77it/s]
144it [00:09, 15.28it/s]
151it [00:09, 21.40it/s]
157it [00:10, 15.92it/s]
160it [00:10, 16.51it/s]
167it [00:10, 23.20it/s]
173it [00:11, 15.97it/s]
180it [00:11, 21.60it/s]
189it [00:12, 16.43it/s]
197it [00:12, 22.17it/s]
204it [00:12, 27.48it/s]
210it [00:13, 16.23it/s]
219it [00:13, 22.87it/s]
225it [00:14, 15.84it/s]
231it [00:14, 19.62it/s]
237it [00:14, 14.90it/s]
241it [00:14, 16.63it/s]
248it [00:15, 22.23it/s]
253it [00:15, 13.35it/

2152it [01:10, 67.19it/s]
2162it [01:10, 75.06it/s]
2170it [01:10, 55.41it/s]
2178it [01:10, 60.00it/s]
2187it [01:10, 66.26it/s]
2196it [01:10, 70.75it/s]
2204it [01:11, 66.47it/s]
2213it [01:11, 71.01it/s]
2221it [01:11, 53.26it/s]
2229it [01:11, 58.52it/s]
2239it [01:11, 67.50it/s]
2247it [01:11, 68.30it/s]
2257it [01:11, 76.20it/s]
2266it [01:12, 54.24it/s]
2274it [01:12, 59.49it/s]
2283it [01:12, 65.42it/s]
2293it [01:12, 72.22it/s]
2302it [01:12, 75.44it/s]
2311it [01:12, 76.44it/s]
2320it [01:12, 77.09it/s]
2329it [01:12, 78.78it/s]
2338it [01:13, 80.05it/s]
2347it [01:13, 82.26it/s]
2357it [01:13, 85.07it/s]
2367it [01:13, 87.81it/s]
2377it [01:13, 87.75it/s]
2387it [01:13, 89.69it/s]
2397it [01:13, 90.22it/s]
2407it [01:13, 90.92it/s]
2417it [01:13, 91.03it/s]
2427it [01:14, 89.85it/s]
2436it [01:14, 86.92it/s]
2445it [01:14, 86.21it/s]
2454it [01:14, 85.82it/s]
2463it [01:14, 84.92it/s]
2472it [01:14, 85.40it/s]
2482it [01:14, 87.23it/s]
2493it [01:14, 90.45it/s]
2503it [01:1

Epoch:  27  loss:  0.5527827702113353



0it [00:00, ?it/s]
1it [00:00,  1.58it/s]
5it [00:01,  2.88it/s]
12it [00:01,  8.19it/s]
20it [00:02, 15.47it/s]